In [ ]:
import pandas as pd
import numpy as np
import random
import glob
import tempfile
import cv2
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.preprocessing import LabelEncoder
import torch
from torch import nn, optim, Tensor
from torch.optim.lr_scheduler import CosineAnnealingLR
from torchinfo import summary
import timm
import torchvision
from torchvision import transforms, datasets, models
import torch.utils.data as Data
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, random_split
from matplotlib import pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [ ]:
train = pd.read_csv(r"/kaggle/input/UBC-OCEAN/train.csv")
test = pd.read_csv(r"/kaggle/input/UBC-OCEAN/test.csv")
train_tma = train[train["is_tma"] == True]
train_no_tma = train[train["is_tma"] == False]

In [ ]:
train_tma['img_id_ext'] = [str(i) + " .png" for i in train_tma['image_id']]
train_no_tma['img_id_ext'] = [str(i) + "_thumbnail.png" for i in train_no_tma['image_id']]
test['img_id_ext'] = [str(i) + "_thumbnail.png" for i in test['image_id']]

In [ ]:
train_df = pd.concat([train_tma, train_no_tma])
train_df.sort_index(ascending = True, inplace = True)
train_df.head()

In [ ]:
le = LabelEncoder()
image_label = train_df['label']
le.fit(image_label)
train_df['label'] = le.transform(image_label)
train_df.head()